In [ ]:
import pandas as pd
import numpy as np

# Abrir .csv con codificación Latin-1
df = pd.read_csv('Establecimientos DEIS MINSAL 01-07-2025(B ESTABLECIMIENTO_2025-07-01).csv', sep=';', encoding='latin-1', skiprows=1)
#nombres de las columnas
#df.columns.to_list()
# usar df con las columnas que nos interesan
df = df[['Nombre Oficial', 'Dirección', 'Tipo Establecimiento (Unidad)', 'Nombre Comuna', 'Teléfono','LATITUD [Grados decimales]', 'LONGITUD [Grados decimales]','Nombre Región' ]]
df = df.rename(columns={
    'Nombre Oficial': 'nombre',
    'Dirección': 'direccion',
    'Tipo Establecimiento (Unidad)': 'tipo_centro',
    'Nombre Comuna': 'comuna',
    'Teléfono': 'telefono',
    'LATITUD [Grados decimales]': 'latitud',
    'LONGITUD [Grados decimales]': 'longitud',
    'Nombre Región': 'region'
})
df = df[['nombre', 'direccion', 'latitud', 'longitud', 'tipo_centro', 'telefono', 'comuna', 'region']]
#Por ahora dejar solo en región del Maule, quitar esto para sacar todo el pais
df = df[df['region'] == 'Región Del Maule']
df = df.drop(columns=['region'])
# Si latitud o longitud dice Pendiente poner null
df['latitud'] = df['latitud'].replace('Pendiente', np.nan)
df['longitud'] = df['longitud'].replace('Pendiente', np.nan)

df['latitud'] = df['latitud'].astype(str).str.replace(',', '.').astype(float)
df['longitud'] = df['longitud'].astype(str).str.replace(',', '.').astype(float)

# Agregar columnas faltantes
df['horario'] = ''
df['servicio_dialisis'] = False
df['activo'] = True
df

,nombre,direccion,latitud,longitud,tipo_centro,telefono,comuna,horario,servicio_dialisis,activo
1604,Posta de Salud Rural Villa Baviera,Camino a Catillo,-36.387935,-71.585985,Posta de Salud Rural (PSR),NaN,Parral,,False,True
1605,Laboratorio Clínico Perquilauquén,"Parcela, Villa Baviera",-36.387277,-71.588431,Laboratorio Clínico,NaN,Parral,,False,True
1612,Posta de Salud Rural Los Canelos (Parral),Caserío Los Canelos,-36.303463,-71.318120,Posta de Salud Rural (PSR),NaN,Parral,,False,True
1614,Posta de Salud Rural Bullileo,Caserío Bullileo,-36.288958,-71.414922,Posta de Salud Rural (PSR),NaN,Parral,,False,True
1615,Posta de Salud Rural Bajos de Huenutil,Caserío Bajos de Huenutil,-36.288946,-71.741127,Posta de Salud Rural (PSR),NaN,Parral,,False,True
...,...,...,...,...,...,...,...,...,...,...
4820,Vacunatorio San José,Freire,NaN,NaN,Vacunatorio,997808815,Linares,,False,True
4821,Vacunatorio Privado Amigable,Las Rastras,NaN,NaN,Vacunatorio,936261842,Talca,,False,True
4858,Tripanko,Portales 322-B,NaN,NaN,Centro Radiológico,995630766,Constitución,,False,True
4918,Kineciamed,Merced,NaN,NaN,Centro de Salud Privado,75 2 220133,Curicó,,False,True


In [40]:
import psycopg2
from psycopg2 import sql
import uuid

# Conexión a la base de datos
conn = psycopg2.connect(
    host="localhost",
    database="BD_SAFETYIRC",
    user="postgres",
    password="1234",
    port="5432"
)
cur = conn.cursor()

# Consulta de inserción
insert_query = """
INSERT INTO api_centromedico (
    id, nombre, direccion, latitud, longitud, tipo_centro,
    telefono, horario, servicio_dialisis, activo, comuna
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insertar cada fila
for index, row in df.iterrows():
    cur.execute(insert_query, (
        str(uuid.uuid4()),  # id
        row['nombre'],
        row['direccion'],
        row['latitud'],
        row['longitud'],
        row['tipo_centro'],
        row['telefono'],
        row['horario'],
        row['servicio_dialisis'],
        row['activo'],
        row['comuna']
    ))

# Guardar y cerrar
conn.commit()
cur.close()
conn.close()